In [1]:
import dai
import pandas as pd

import sys
sys.path.append("/home/aiuser/work/GitHub/BigQuant_Resources_Collection/BigQuant_Research_Frameworks/General_Tools")
sys.path.append("/home/aiuser/work/GitHub/BigQuant_Resources_Collection/BigQuant_Research_Frameworks/Strategy_Tools")
from General_Tools import *
from Stock_Factor_Strategy_Tools import *

In [2]:
sd = get_date_str('2019-12-31')
ed = get_date_str('2026-02-13')

In [3]:
sql_factor =  f"""
SELECT
    date,
    instrument,
    dividend_yield_ratio AS factor
FROM cn_stock_prefactors
QUALIFY c_pct_rank(total_market_cap) > 0.20
AND c_pct_rank(pe_ttm) < 0.40
AND pe_ttm > 0 
AND ps_ttm < 2.5
"""

In [4]:
sql_trade = get_trade_sql(sql_factor, number_security=10, position_allocation="equal", rebalance_period="month", stock_pool="000300.SH")

In [5]:
strategy_df = dai.query(sql_trade, filters = {"date":[sd, ed]}).df()
strategy_df

,date,instrument,score,score_rank,position
0,2019-12-31,600188.SH,0.145833,1.0,0.1
1,2019-12-31,600019.SH,0.087108,2.0,0.1
2,2019-12-31,600028.SH,0.074364,3.0,0.1
3,2019-12-31,000898.SZ,0.065672,4.0,0.1
4,2019-12-31,601006.SH,0.058465,5.0,0.1
...,...,...,...,...,...
735,2026-01-30,000157.SZ,0.057013,6.0,0.1
736,2026-01-30,601166.SH,0.056684,7.0,0.1
737,2026-01-30,601229.SH,0.056277,8.0,0.1
738,2026-01-30,000001.SZ,0.055217,9.0,0.1


In [6]:
df1 = backtest_factor_strategy(strategy_df, capital_base=1000000)

[2026-02-17 16:56:31] [info     ] bigtrader.v34 开始运行 ..
[2026-02-17 16:56:31] [info     ] 2019-12-31, 2026-01-30, , , instruments=78
[2026-02-17 16:56:31] [info     ] bigtrader module V2.2.0
[2026-02-17 16:56:31] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-17 16:56:35] [info     ] backtest done, raw_perf_ds:dai.DataSource("_deb855ef3159402aa07b2a3035677625")


[2026-02-17 16:56:36] [info     ] bigtrader.v34 运行完成 [5.479s].


In [7]:
df2 = backtest_market_neutral_strategy(strategy_df, capital_base=1000000)

[2026-02-17 16:56:37] [info     ] bigtrader.v34 开始运行 ..
[2026-02-17 16:56:37] [info     ] 2019-12-31, 2026-01-30, , future, instruments=78
[2026-02-17 16:56:37] [info     ] bigtrader module V2.2.0
[2026-02-17 16:56:37] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-17 16:56:38] [info     ] backtest done, raw_perf_ds:dai.DataSource("_0707f760eac1430ab16a57462476c9aa")


[2026-02-17 16:56:39] [info     ] bigtrader.v34 运行完成 [2.076s].


In [8]:
df1_new = df1[['date','portfolio_value']].rename(columns={'portfolio_value': 'portfolio_value_stock'}, inplace=False).reset_index(drop=True)
df2_new = df2[['date','portfolio_value']].rename(columns={'portfolio_value': 'portfolio_value_index'}, inplace=False).reset_index(drop=True)

In [9]:
df_merge = pd.merge(df1_new, df2_new, on='date', how='inner')
df_merge['portfolio_value']       = df_merge['portfolio_value_stock'] + df_merge['portfolio_value_index']
df_merge['portfolio_value']       = df_merge['portfolio_value']       / df_merge['portfolio_value'].iloc[0]
df_merge['portfolio_value_stock'] = df_merge['portfolio_value_stock'] / df_merge['portfolio_value_stock'].iloc[0]
df_merge['portfolio_value_index'] = df_merge['portfolio_value_index'] / df_merge['portfolio_value_index'].iloc[0]

df_merge

,date,portfolio_value_stock,portfolio_value_index,portfolio_value
0,2019-12-31,1.000000,1.000000,1.000000
1,2020-01-02,0.999701,0.991212,0.995456
2,2020-01-03,1.000354,0.993792,0.997073
3,2020-01-06,0.994588,0.999551,0.997070
4,2020-01-07,1.000669,0.990612,0.995640
...,...,...,...,...
1471,2026-01-26,1.703665,0.819347,1.261505
1472,2026-01-27,1.683347,0.819707,1.251526
1473,2026-01-28,1.690937,0.815327,1.253131
1474,2026-01-29,1.710313,0.799967,1.255139


In [10]:
from bigcharts import Line
Line(df_merge)